# Visualizing the installed pv power per area for urban and rural communes in Germany

# I Introduction

In this script we will try to get an overview of the installed power from photovoltaics per area in units of $[\frac{kWp}{km^2}]$ for every administrative district in Germany, thus providing us with an interactive heatmap, showing low and high concentrations of pv penetration over the country as a whole.

**NOTE:** Interest lies in residential pv plants, therefore the following analysis will concentrate on those pv plants that are connected to the low- and medium-voltage power grid only.

The data used:
- the asset master data, provided by the 4 german TSOs on [netztransparenz.de](https://www.netztransparenz.de/EEG/Anlagenstammdaten) for the year 2019 
- the geographic boundary data for every administrative district, available on [opendatalab.de](http://opendatalab.de/projects/geojson-utilities/), with optional data on area
- a mapping table that identifies geo coordinates to every zip code, available on [OpenGeoDB](http://www.fa-technik.adfc.de/code/opengeodb/) 

is saved in the *./data* subfolder.

Lets start by importing the relevant packages.

In [1]:
import branca.colormap as cmp #define a color gradient manually 
import folium #manipulate data and visualize on a Leaflet map 
from folium.map import *
from folium.plugins import MeasureControl
import geojson #load polygonal boundary data
import joblib #load pickled files
import numpy as np 
import os.path 
import pandas as pd 
from shapely.geometry import Point,shape #filter geo coordinates based on polygonal areas 
from tqdm import tqdm_notebook #progress bar

working_dir = './data/' #directory of input data
result_dir = './results/' #directory of resulting html

## II Data preparation

We load the asset master data into the running session, adjust their column names and concatenate into one dataframe. 

In [2]:
#load csv data from every TSO
df_trans = pd.read_csv(working_dir+'TransnetBW GmbH Anlagenstammdaten 2019.csv',sep=';',engine='python',decimal=',') 
df_50 = pd.read_csv(working_dir+'50Hertz Transmission GmbH EEG-Zahlungen Stammdaten 2019.csv',sep=';',engine='python',decimal=',')
df_ampr = pd.read_csv(working_dir+'Amprion GmbH EEG-Zahlungen Anlagenstammdaten 2019.csv',sep=';',engine='python',decimal=',')
df_tenn = pd.read_csv(working_dir+'Netztransparenz Anlagenstammdaten 2019 TenneT TSO GmbH.csv',sep=';',engine='python',decimal=',')

#adjust column names (i.e. same attributes in same order, yet different acronyms...)
df_trans.columns=df_50.columns
df_ampr.columns=df_50.columns
df_tenn.columns=df_50.columns

#concatenate
df = pd.concat([df_trans, df_50, df_ampr, df_tenn], ignore_index=True, sort=False)
df.reset_index(drop=True,inplace=True)

#single csv files are not needed in the following
del df_trans, df_50, df_ampr, df_tenn

The zip codes have been loaded as integer numbers, thus zip codes with a leading zero have been truncated. We need to transform this attribute to string values and add the leading zeros back in. 

In [3]:
plz=df['PLZ'].astype(str)
for idx in range(len(plz)):
    if len(plz[idx])==4:
        plz[idx]='0'+plz[idx]
        
df['PLZ']=plz        

The dataframe contains data on all kinds of energy sources. Next, we need to filter for photvoltaic and for the low- and medium voltage (lvmv) power grid and sum up the total installed pv power per zip code, resulting in a more aggregated and suitable form in the following steps.  

In [4]:
#filter for pv
df_pv = df.loc[df['Energietraeger']=='Solar',:]

#filter for low voltage/medium voltage (lvmv), groupby zip code and installed power and use summation as aggregation method 
df_pv_aggr = df_pv.loc[df_pv['Einspeisespannungsebene'].isin(['NS','MS','MS/NS']),
                       ['PLZ','Installierte_Leistung']].groupby('PLZ').sum()
df_pv_aggr.reset_index(inplace=True)


To that new table we add the geo-coordinates associated with each zip code. For that, we load the mapping table and adjust again for missing zeros in the postal code before adding columns for longitude and latitude to the aggregated dataframe.

In [5]:
#load the mapping table
plz_file = pd.read_csv(working_dir+'PLZ.csv',engine='python',sep=';')

#adjust for leading zero in zip doe
plz=plz_file['plz'].astype(str)
for idx in range(len(plz)):
    if len(plz[idx])==4:
        plz[idx]='0'+plz[idx]
        
plz_file.index=plz

#add new columns for longitude/latitude
df_pv_aggr['lon']=0
df_pv_aggr['lat']=0

#do the mapping
for idx in range(df_pv_aggr.shape[0]):
    if df_pv_aggr.loc[idx,'PLZ'] in plz_file.index:
        df_pv_aggr.loc[idx,'lon'] = plz_file.loc[df_pv_aggr.loc[idx,'PLZ'],'lon']
        df_pv_aggr.loc[idx,'lat'] = plz_file.loc[df_pv_aggr.loc[idx,'PLZ'],'lat']

Lastly, we load the .geojson file. Each feature contains the polygonal boundaries (and additional data such as the area) for a specified rural or urban commune. We need to address each feature for the final representation, so let us add an attribute *id* to each feature.   

In [7]:
#open file
with open(working_dir+'gemeinden_simplify20.geojson') as f:
    gj_ad = geojson.load(f)

#add index to each feature    
for idx in (range(len(gj_ad['features']))):
    gj_ad['features'][idx]['id']=idx 

We want to know the installed power per area for every administrative district. In an intermediate step, we have to sum the installed power for the local districts that fall under the respective administration. Thus, what we need is a relation between the vector of installed power for the adminstrative districts $\vec{P}_{ad}$ and the local ones, specified by their zip code $\vec{P}_{zip}$, given by the vector matrix equation

$$\vec{P}_{ad} = M_{zip \to ad}\vec{P}_{zip}$$

The matrix $M_{zip \to ad}$ does the mapping and has elements $(m)_{ij}\in\{0,1\}$ which needs to be specified now.
To this end, we check for each polygonal boundary of every commune if the geo-coordinates, associated with each zip-code, lie within it (therefore filling the matrix with ones and zeroes).

In [8]:
#the following code block is rather resourceful, so the precomputed mapping matrix is added to the repository

if os.path.isfile(working_dir+'mapping_lvmv.pkl'):
    #load precomputed mapping matrix
    mapping_lvmv=joblib.load(working_dir+'mapping_lvmv.pkl')
    
else:
    #initialize empty matrix
    mapping_lvmv = np.zeros((len(gj_ad['features']),df_pv_aggr.shape[0]))
    
    for i in tqdm_notebook(range(len(gj_ad['features']))):
        #load the polygon of each commune
        polygon = shape(gj_ad['features'][i]['geometry'])
        for j in range(df_pv_aggr.shape[0]):        
            point = Point(df_pv_aggr.loc[j,'lon'],df_pv_aggr.loc[j,'lat'])
            #check if the geo-coordinates associated with each zip code lie within it
            if point.within(polygon):
                mapping_lvmv[i,j] = 1 
    
    #save resulting mapping table
    joblib.dump(mapping_lvmv,working_dir+'mapping_lvmv.pkl')

The installed power of the $i$th administrative district is now given by

$$(\vec{P_{ad}})_{i} = \sum_{j}m_{ij}(\vec{P}_{zip})_{j}$$

and the installed power per area can be derived without further ado.

In [9]:
#get the installed power per district through matrix vector multiplication 
pv_installed_lvmv = pd.DataFrame(mapping_lvmv.dot(df_pv_aggr['Installierte_Leistung'].values)).reset_index()
pv_installed_lvmv.columns=['ID','Installed']

#divide by area to get installed power per area        
pv_installed_pa = pv_installed_lvmv.copy()
for idx in range(len(gj_ad['features'])):
    #check if the feature contains further data
    if 'destatis' in gj_ad['features'][idx]['properties'].keys():
        #if so, devide by the area through proper indexation
        pv_installed_pa.loc[idx,'Installed'] = pv_installed_lvmv.loc[idx,'Installed']/gj_ad['features'][idx]['properties']['destatis']['area']
    else:
        #otherwise, set value to zero (see following commend)
        print('Not included: ' + gj_ad['features'][idx]['properties']['GEN'])
        pv_installed_pa.loc[idx,'Installed'] = 0

Not included: Sachsenwald (Forstgutsbez.)
Not included: Buchholz (Forstgutsbez.)
Not included: Giebel
Not included: Harz (Landkreis Goslar)
Not included: Brunsleberfeld
Not included: Helmstedt
Not included: KÃ¶nigslutter
Not included: Mariental
Not included: SchÃ¶ningen
Not included: Solling (Landkreis Northeim)
Not included: Am GroÃŸen Rhode
Not included: Barnstorf-Warle
Not included: Voigtsdahlum
Not included: Harz (Landkreis GÃ¶ttingen)
Not included: Boffzen
Not included: Eimen
Not included: Eschershausen
Not included: GrÃ¼nenplan
Not included: Holzminden
Not included: Merxhausen
Not included: Wenzen
Not included: Gartow
Not included: GÃ¶hrde
Not included: Nordseeinsel Memmert
Not included: Insel LÃ¼tje HÃ¶rn
Not included: Michelbuch
Not included: Gutsbezirk Spessart
Not included: Gutsbezirk Reinhardswald
Not included: Gutsbezirk Kaufunger Wald
Not included: Rheinau
Not included: Gutsbezirk MÃ¼nsingen
Not included: Eck
Not included: Schellenberger Forst
Not included: Pupplinger Au
N

As can be seen, there is no data on the area for some small districts, which have been omitted in the final representation (i.e. their installed power per area has been set to zero). The impact of this simplification is assumed to be negligible.  

## III Visualization

The [folium](https://python-visualization.github.io/folium/) package is used to create an interactive Leaflet map and export it as an html file. 

The default option for creating a heatmap based on polygonmal data and a dictionary or series of respective values allows only for a discrete coloring scheme of six colors. This is unfavourable, as this resolution is to rough. Instead, we want to use a continuous color gradient. In order to avoid some very few high values to skew the coloring, distinct colors are only mapped up to the 90% quantile of the data.

In [28]:
#specify a continuous color gradient
linear_pa = cmp.LinearColormap(
    ['yellow', 'yellowgreen', 'green', 'indigo'],
    #avoid skewing the coloring and define the gradient only up to the 90% quantile
    vmin=min(pv_installed_pa['Installed']), vmax=pv_installed_pa['Installed'].quantile(0.9),
    caption='Installed power per area in [kWp/km^2]'
)

#define a series with the same indexing as the boundary data gj_ad
pv_dict_pa = pv_installed_pa.set_index('ID')['Installed']

#draw the map for Germany
mDE_pa = folium.Map(location=(51.165691, 10.451526), zoom_start=7,tiles='CartoDBPositron')

#add the heatmap
folium.GeoJson(
    gj_ad,
    style_function=lambda feature: {
        'fillColor': linear_pa(pv_dict_pa[feature['id']]),
        'color': 'red',
        'weight': 0.1        
    }
).add_to(mDE_pa)
linear_pa.add_to(mDE_pa)

folium.LayerControl().add_to(mDE_pa)
mDE_pa.add_child(MeasureControl())

mDE_pa
#save as html file
mDE_pa.save(result_dir+'installed_power_LVMV_per_area_2019.html')

The final visualization is saved in the *./results* subfolder. We observe a high concentration of installed power per area in the south of Germany. 